In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, LayerNormalization, Embedding, LSTM, GRU, Dense, Bidirectional, TimeDistributed, Input, Dropout
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard, LambdaCallback
import pandas as pd
import numpy as np
import re
import itertools
import random
# from keras_nlp.layers import PositionEmbedding, TokenAndPositionEmbedding
import slot_util as su
import sentence_from_template as sft
from tokenizer import Tokenizer
from data_generator import DataGenerator
import pickle


In [ ]:
# slots: num_room, time, time_unit, num_bed, num_person, person_unit, hotel_name, hotel_city,
# slots: person_unit_post, hotel_name_post, hotel_city_post
"''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''"
# Fillers: for, me, for_me, from, verb, please, in, bed_unit
"''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''"
# {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {} {}
"''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''"
# {num_room} {time}
# {time_unit} {num_bed}
# {num_person} {person_unit}
# {hotel_name} {hotel_city}
"''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''"
# {verb} {for}
# {for_{me}} {me}
# {from} {please}
# {in} {bed_unit}


In [27]:
# TRAINING  PARAMETERS
BATCH_SIZE = 32
steps_per_epoch=5000
validation_steps=3000
EPOCHS = 3

# EMBEDDING  PARAMETERS
NUM_WORDS = 300
MAX_SEQ_LEN = 45
EMBEDDING_DIM = 300
VOCAB_SIZE = NUM_WORDS + 50


# ENCODER/RNN  PARAMETERS
INTERMEDIATE_DIM = 200
NUM_HEADS = 2
NUM_UNITS = 150

In [10]:
TEST_SET = False
SPLIT = 0.2
with open("Hotel.txt", encoding='utf-8') as f:
    templates = f.readlines()
    templates = [line.strip().split(" ") for line in templates]

random.shuffle(templates)
num_val_samples = int(SPLIT * len(templates))
if TEST_SET:
    num_train_samples = len(templates) - 2 * num_val_samples
    train_templates = templates[:num_train_samples]
    val_templates = templates[num_train_samples : num_train_samples + num_val_samples]
    test_templates = templates[num_train_samples + num_val_samples :]
else:
    num_train_samples = len(templates) - num_val_samples
    train_templates = templates[:num_train_samples]
    val_templates = templates[num_train_samples : num_train_samples + num_val_samples]
    test_templates = []


print(f"{len(templates)} total templates")
print(f"{len(train_templates)} training templates")
print(f"{len(val_templates)} validation templates")
print(f"{len(test_templates)} test templates")


# Uncomment if you want to load saved splited templates
# with open("train_data.pickle", "rb") as f:
#     train_templates = pickle.load(f)
# with open("validation_data.pickle", "rb") as f:
#     val_templates = pickle.load(f)
# if TEST_SET:
#     with open("test_data.pickle", "rb") as f:
#         test_templates = pickle.load(f)

72 total templates
58 training templates
14 validation templates
0 test templates


In [4]:
for t in val_templates[:2]:
    print(t)

['{in}', 'هتل', '{hotel_name}']
['برو', '{for_{me}}', 'توی', 'شهر', '{hotel_city}', '{for}', '{time}', '{time_unit}', 'اتاق', '{num_bed}', '{bed_unit}', '{in}', 'هتل', '{hotel_name}', '{verb}']


In [28]:
x_tokenizer = Tokenizer(num_words=NUM_WORDS, max_seq_len=MAX_SEQ_LEN,
                        for_sentence=True, sos=False, eos=False)
y_tokenizer = Tokenizer(num_words=NUM_WORDS, max_seq_len=MAX_SEQ_LEN,
                        for_sentence=False, sos=False, eos=False)

train_data_gen = DataGenerator(train_templates,
                               x_tokenizer, y_tokenizer,
                               BATCH_SIZE, steps_per_epoch,
                               aug_percent=0.0, max_seq_len=MAX_SEQ_LEN)

val_data_gen = DataGenerator(val_templates,
                             x_tokenizer, y_tokenizer,
                             BATCH_SIZE, validation_steps,
                             aug_percent=0.0, max_seq_len=MAX_SEQ_LEN)
if TEST_SET:
    test_data_gen = DataGenerator(test_templates,
                                  x_tokenizer, y_tokenizer,
                                  BATCH_SIZE, validation_steps,
                                  aug_percent=0.0, max_seq_len=MAX_SEQ_LEN)

Y_VOCAB_SIZE = len(y_tokenizer.word_to_index)

all words length =  123


In [29]:

for _ in range(1000):
    X, y = train_data_gen.__getitem__(0)

In [30]:
X

array([[302,  75, 302, ...,   0,   0,   0],
       [108, 302, 302, ...,   0,   0,   0],
       [ 40, 108, 302, ...,   0,   0,   0],
       ...,
       [111,  84,  39, ...,   0,   0,   0],
       [302,   4,  71, ...,   0,   0,   0],
       [168,   4,   0, ...,   0,   0,   0]])

In [31]:
x_tokenizer.seq_to_words(X[6])

['هتل',
 'جبظخفظثدوو',
 'جبظخفظثدوو',
 'جبظخفظثدوو',
 'جبظخفظثدوو',
 'شهر',
 'جبظخفظثدوو',
 'جبظخفظثدوو',
 'جبظخفظثدوو']

In [26]:
for t in train_templates:
    print(t)

['{for}', '{num_person}', '{person_unit}', 'و', '{num_person}', '{person_unit}', 'و', '{time}', '{time_unit}', 'و', '{time}', '{time_unit}', '{in}', 'هتل', '{hotel_name}', 'شهر', '{hotel_city}', 'اتاق', '{verb}', '{for_{me}}']
['{from}', 'هتل', '{hotel_name}', '{in}', 'شهر', '{hotel_city}', '{for}', '{time}', '{time_unit}', 'و', '{time}', '{time_unit}', '{num_room}', 'اتاق', '{num_bed}', '{bed_unit}', '{verb}']
['یه', 'اتاق', '{num_person}', '{person_unit}', '{in}', 'شهر', '{hotel_city}', 'هتل', '{hotel_name}', '{verb}']
['{num_room}', 'اتاق', '{in}', 'هتل', '{hotel_name}', '{for}', '{num_person}', '{person_unit}', 'و', '{time}', '{time_unit}', 'شهر', '{hotel_city}', '{verb}']
['{for_{me}}', 'توی', 'شهر', '{hotel_city}', 'اتاق', '{num_bed}', '{bed_unit}', '{verb}']
['{num_room}', 'اتاق']
['هتل', '{hotel_name}', '{for}', '{time}', '{time_unit}', '{in}', 'شهر', '{hotel_city}', '{for_{me}}', 'اتاق', '{verb}']
['{num_room}', 'اتاق', '{in}', 'هتل', '{hotel_name}', 'شهر', '{hotel_city}', '{f